In [21]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer


In [23]:
datasets = {
    'us_crime': "https://archive.ics.uci.edu/static/public/183/data.csv",
    'letter_img': "https://archive.ics.uci.edu/static/public/59/data.csv",
    'mammography': "https://archive.ics.uci.edu/static/public/161/data.csv"
}

data_dict = {}
for name, url in datasets.items():
    data = pd.read_csv(url, header=None)

    if name == 'us_crime':
        data = data.drop(columns=[3])
        X = data.iloc[1:, :-1]
        y = data.iloc[1:, -1].astype(float).apply(lambda x: 1 if x > 0.65 else 0).astype(int)  # Target is the last column
    elif name == 'letter_img':
        X = data.iloc[1:, 1:]  # Features start from the second column
        y = data.iloc[1:, 0].apply(lambda x: 1 if x == 'Z' else 0).astype(int)   # Target is the first column
    elif name == 'mammography':
        X = data.iloc[1:, :-1]
        y = data.iloc[1:, -1].astype(int)  # Target is the last column
    data_dict[name] = (X, y)


In [33]:
def calculate_f1_scores_multiple_splits(X, y, num_splits= 10):
    # multiple splits (tests) for having a more reliable and accurate features.
    f1_scores = []
    for _ in range(num_splits):
      random_state = np.random.randint(100) # Generate a random seed
      for i in range(X.shape[1]):
          X_feature = X.iloc[:, i].values.reshape(-1, 1)

          # Convert '?' to NaN and then impute missing values
          X_feature = np.where(X_feature == '?', np.nan, X_feature)
          imputer = SimpleImputer(strategy='median') # Or another suitable strategy
          X_feature = imputer.fit_transform(X_feature)

          X_train, X_test, y_train, y_test = train_test_split(X_feature, y, test_size=0.25, random_state= random_state) #random_state for having the mix of examples.


          clf = DecisionTreeClassifier()
          clf.fit(X_train, y_train)
          y_pred = clf.predict(X_test)

          average = 'binary'
          f1 = f1_score(y_test, y_pred, average= average)
          f1_scores.append(f1)

    return np.array(f1_scores).reshape(num_splits, -1) # Reshape for easy averaging


In [34]:
def evaluate_svm_with_selected_features_multiple_splits(X, y, feature_ranking, fractions, num_splits= 10):
    results_all= []
    for _ in range(num_splits):
      random_state= np.random.randint(100)
      results = {'fraction': [], 'f1_score': [], 'precision': [], 'recall': []}
      for fraction in fractions:
          num_features = max(1, int(fraction * X.shape[1])) # ensure that at least one feature is selected.
          selected_features = feature_ranking[:num_features]
          X_selected = X.iloc[:, selected_features]

          # Convert '?' to NaN and then impute missing values
          X_selected = np.where(X_selected == '?', np.nan, X_selected)
          imputer = SimpleImputer(strategy='median') #most_frequent
          X_selected = imputer.fit_transform(X_selected)

          X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.25, random_state= random_state)

          scaler = StandardScaler()
          X_train = scaler.fit_transform(X_train)
          X_test = scaler.transform(X_test)

          svm = SVC(kernel='rbf', class_weight= 'balanced')
          svm.fit(X_train, y_train)
          y_pred = svm.predict(X_test)

          average = 'binary'
          f1 = f1_score(y_test, y_pred, average= average)
          precision = precision_score(y_test, y_pred, average= average)
          recall = recall_score(y_test, y_pred, average= average)

          results['fraction'].append(fraction)
          results['f1_score'].append(f1)
          results['precision'].append(precision)
          results['recall'].append(recall)
      results_all.append(results)
    return results_all


In [ ]:
feature_ranking
# f1_scores

[0.798076923076923,
 0.798076923076923,
 0.798076923076923,
 0.7981220657276995,
 0.7981220657276995,
 0.8389830508474576,
 0.8389830508474576,
 0.8189655172413793,
 0.8189655172413793,
 0.8189655172413793]

In [ ]:
fractions = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for name, (X, y) in data_dict.items():
    print(f"Evaluating dataset: {name}")
    print(f"  X shape: {X.shape}")
    print(f"  y shape: {y.shape}")
    unique, counts = np.unique(y, return_counts=True)
    ratio = int(counts[np.argmax(counts)] / counts[np.argmin(counts)])
    print(f"  Ratio: {ratio}:1")
    # print("\n")

    f1_scores_multiple = calculate_f1_scores_multiple_splits(X, y)
    f1_scores= f1_scores_multiple.mean(axis=0) # Average across rows (splits)

    # Rank features based on F1-scores
    feature_ranking = np.argsort(f1_scores)[::-1]

    # Evaluate SVM classifier with selected features
    results_multiple = evaluate_svm_with_selected_features_multiple_splits(X, y, feature_ranking, fractions)

    # Calculate average scores for each fraction
    average_f1_scores = []
    average_precision_scores = []
    average_recall_scores = []

    for fraction in fractions:
        f1_scores_for_fraction = [result['f1_score'][fractions.index(fraction)] for result in results_multiple]
        precision_scores_for_fraction = [result['precision'][fractions.index(fraction)] for result in results_multiple]
        recall_scores_for_fraction = [result['recall'][fractions.index(fraction)] for result in results_multiple]

        average_f1_scores.append(np.mean(f1_scores_for_fraction))
        average_precision_scores.append(np.mean(precision_scores_for_fraction))
        average_recall_scores.append(np.mean(recall_scores_for_fraction))

    # Now we have average scores for each fraction across multiple splits
    # Use these average scores for plotting instead of the single-split scores
    f1_scores = average_f1_scores
    recall_scores = average_recall_scores
    precision_scores = average_precision_scores
    # # Extract the scores for plotting
    # f1_scores = results['f1_score']
    # recall_scores = results['recall']
    # precision_scores = results['precision']

    # Plot the results
    plt.figure(figsize=(15, 5))

    # Plot F1-score
    plt.subplot(1, 3, 1)
    plt.plot(fractions[: -1], f1_scores[: -1], color= "blue")
    plt.axhline(y= f1_scores[-1], xmin= 0, xmax= 1.0+ 0.05,
            label='Full set', color="red", linestyle='--')

    plt.title(f'F1-score ({name})')
    plt.xlabel('Fraction of Features')
    plt.ylabel('F1-score')
    plt.grid(True, color='white')
    plt.gca().set_facecolor('lightblue')
    plt.grid(True)
    plt.legend()

    # Plot Recall score
    plt.subplot(1, 3, 2)
    plt.plot(fractions[: -1], recall_scores[: -1], color= "blue")
    plt.axhline(y= recall_scores[-1], xmin= 0, xmax= 1.0+ 0.05,
            label='Full set', color="red", linestyle='--')

    plt.title(f'Recall ({name})')
    plt.xlabel('Fraction of Features')
    plt.ylabel('Recall Score')
    plt.grid(True, color='white')
    plt.gca().set_facecolor('lightblue')
    plt.grid(True)
    plt.legend()

    # Plot Precision score
    plt.subplot(1, 3, 3)
    plt.plot(fractions[: -1], precision_scores[: -1], color= "blue")
    plt.axhline(y= precision_scores[-1], xmin= 0, xmax= 1.0+ 0.05,
            label='Full set', color="red", linestyle='--')

    plt.title(f'Precision ({name})')
    plt.xlabel('Fraction of Features')
    plt.ylabel('Precision Score')
    plt.grid(True, color='white')  # White grid lines
    plt.gca().set_facecolor('lightblue')  # Light blue background
    plt.grid(True)

    plt.tight_layout()
    plt.legend()
    plt.show()

    print("The best fraction of features in this problem based on the F1-score is", fractions[f1_scores.index(max(f1_scores))])
    print("Therefore we can use the above fraction as an optima.")
    print(20* '*', '\n')

Evaluating dataset: us_crime
  X shape: (1994, 126)
  y shape: (1994,)
  Ratio: 12:1


# **This shows that we do not necessarily need to use all the features for having a good ML model.**